In [1]:
import cv2
import os

#dir=r"C:\Users\Barath\Desktop\Data science\computer vision\capture\train\0"
#record=cv2.VideoCapture("0.mp4")
i=0
while(record.isOpened()):
    ret, frame = record.read()
    if(ret==False):
        break
    cv2.imwrite((os.path.join(dir, str(i)+".jpg")),frame)
    i=i+1
record.release()
cv2.destroyAllWindows()

In [4]:
base_dir = r'C:\Users\Barath\Desktop\Data science\computer vision\capture'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')


train_0_dir = os.path.join(train_dir, "0")
train_1_dir = os.path.join(train_dir, "1")


validation_0_dir = os.path.join(validation_dir, "0")
validation_1_dir = os.path.join(validation_dir, "1")

In [5]:
print(0,len(os.listdir(train_0_dir)))
print(1,len(os.listdir(train_1_dir)))

print("0 Validation", len(os.listdir(validation_0_dir)))
print("1 Validation", len(os.listdir(validation_1_dir)))

0 5460
1 5842
0 Validation 2683
1 Validation 2195


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator(rescale = 1.0/255.)
test_datagen  = ImageDataGenerator(rescale = 1.0/255.)

train_generator=train_datagen.flow_from_directory(train_dir,batch_size=20,class_mode='binary',target_size=(512, 288))     

validation_generator=test_datagen.flow_from_directory(validation_dir,batch_size=20,class_mode='binary',target_size=(512, 288))

Found 11302 images belonging to 2 classes.
Found 4878 images belonging to 2 classes.


In [23]:
remote=tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(4,3),activation="relu",input_shape=(512,288,3)),
    tf.keras.layers.MaxPooling2D(4,3),
    tf.keras.layers.Conv2D(16,(4,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(4,3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')])

In [24]:
remote.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 509, 286, 16)      592       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 169, 95, 16)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 166, 93, 16)       3088      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 55, 30, 16)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 26400)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               3379328   
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [25]:
from tensorflow.keras.optimizers import RMSprop

remote.compile(optimizer=RMSprop(lr=0.003),loss='binary_crossentropy',metrics = ['acc'])

In [26]:
remote.fit_generator(train_generator,validation_data=validation_generator,epochs=15,steps_per_epoch=100,validation_steps=50)

Epoch 1/15
100/100 [==============================] - 208s 2s/step - loss: 2.0523 - acc: 0.5985 - val_loss: 0.4368 - val_acc: 0.7980
Epoch 2/15
100/100 [==============================] - 180s 2s/step - loss: 0.3681 - acc: 0.8770 - val_loss: 0.5058 - val_acc: 0.8330
Epoch 3/15
100/100 [==============================] - 183s 2s/step - loss: 0.1504 - acc: 0.9540 - val_loss: 0.2235 - val_acc: 0.8950
Epoch 4/15
100/100 [==============================] - 193s 2s/step - loss: 0.1575 - acc: 0.9650 - val_loss: 0.2083 - val_acc: 0.9280
Epoch 5/15
100/100 [==============================] - 193s 2s/step - loss: 0.0853 - acc: 0.9825 - val_loss: 0.8587 - val_acc: 0.8230
Epoch 6/15
100/100 [==============================] - 193s 2s/step - loss: 0.2951 - acc: 0.9707 - val_loss: 1.0788 - val_acc: 0.8170
Epoch 7/15
100/100 [==============================] - 200s 2s/step - loss: 0.0468 - acc: 0.9900 - val_loss: 0.9378 - val_acc: 0.8430
Epoch 8/15
100/100 [==============================] - 209s 2s/step - 

In [30]:
remote.save("remote.h5")

In [31]:
from PIL import Image
import numpy as np
model = tf.keras.models.load_model('remote.h5')
video = cv2.VideoCapture("test.mp4")

while True:
    _, frame = video.read()

    #Convert the captured frame into RGB
    im = Image.fromarray(frame)
    
    
    #Resizing into 128x128 because we trained the model with this image size.
    im = im.resize((288,512))
    img_array = np.array(im)

    
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.cast(img_array, tf.float16)
    #Calling the predict method on model to predict 'me' on the image
    prediction = int(remote.predict(img_array)[0][0])

    #if prediction is 0, which mean I am missing on the image, then show the frame in gray color.
    print(prediction)
    if prediction==1:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        label(frame, 'Bottle Detected', (20,20), (255,0,0))
        
    
    cv2.imshow("pred", frame)
    key=cv2.waitKey(1)
    if key == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
1
1
1
1
1
1
0
1
0
0
1
1
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
0
0
1
1
1
1
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


AttributeError: 'NoneType' object has no attribute '__array_interface__'

In [29]:
def label(img, text, pos, bg_color):
    font_face = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1.0
    color = (0, 0, 0)
    thickness = cv2.FILLED
    margin = 2

    txt_size = cv2.getTextSize(text, font_face, scale, thickness)

    end_x = pos[0] + txt_size[0][0] + margin
    end_y = pos[1] - txt_size[0][1] - margin

    cv2.rectangle(img, pos, (end_x, end_y), bg_color, thickness)
    cv2.putText(img, text, pos, font_face, scale, color, 1, cv2.LINE_AA)

In [87]:
test_datagen2  = ImageDataGenerator(rescale = 1.0/255.)
d= r"C:\Users\Barath\Desktop\Data science\computer vision\capture\validation\New folder"
test=test_datagen2.flow_from_directory(d,batch_size=20,class_mode='binary',target_size=(512, 288))
pred=remote.predict(test)

Found 2366 images belonging to 1 classes.


In [65]:
a=pd.DataFrame(pred)

In [64]:
import pandas as pd

In [69]:
a.count_values()

AttributeError: 'DataFrame' object has no attribute 'count_values'

In [32]:
video.release()
cv2.destroyAllWindows()